# Messages and Requests

## Install

In [1]:
SYFT_VERSION = ">=0.8.1b0,<0.9"
package_string = f'"syft{SYFT_VERSION}"'
# %pip install {package_string} -f https://whls.blob.core.windows.net/unstable/index.html

In [2]:
import syft as sy
sy.requires(SYFT_VERSION)


Warning: syft is imported in light mode by default.         
To switch to dark mode, please run `sy.options.color_theme = 'dark'`

✅ The installed version of syft==0.8.1b12 matches the requirement >=0.8.1b0 and the requirement <0.9


In [3]:
node = sy.orchestra.launch(name="messages-requests-example-domain-1",port="auto", reset=True)

Starting messages-requests-example-domain-1 server on 0.0.0.0:4559
SQLite Store Path:
!open file:///tmp/9458ac6a16464e7bb6211f87d19897dd.sqlite

Waiting for server to start... Done.


## Setup

For the purpose of this tutorial we are creating a very simple dataset, which is created and owner by the root client

In [4]:
admin_client = node.login(email="info@openmined.org", password="changethis")

Logged into messages-requests-example-domain-1 as GUEST
Logged into messages-requests-example-domain-1 as <info@openmined.org>


In [5]:
import numpy as np
dataset = sy.Dataset(
    name="my dataset",
    asset_list=[
        sy.Asset(
        name="my asset",
        data=np.array([1,2,3]),
        mock=np.array([1,1,1])
    )]
)
admin_client.upload_dataset(dataset)

100%|██████████| 1/1 [00:00<00:00,  5.20it/s]

Uploading: my asset


100%|██████████| 1/1 [00:00<00:00,  5.18it/s]


<class 'syft.service.response.SyftSuccess'>: Dataset Added

In [6]:
admin_client.register(name="Jane Doe", email="jane@caltech.edu",
                            password="abc123", institution="Caltech", website="https://www.caltech.edu/")

<class 'syft.service.response.SyftSuccess'>: User Jane Doe successfully registered! To see users, run client.users.

In [7]:
guest_client = node.client.login(email="jane@caltech.edu", password="abc123")

Logged into messages-requests-example-domain-1 as GUEST
Logged into messages-requests-example-domain-1 as <jane@caltech.edu>


In [8]:
asset = guest_client.datasets[0].assets[0]

In [9]:
@sy.syft_function_single_use()
def func():
    import random
    data = list(range(100))
    return sum(data) / 100 + random.random()

@sy.syft_function_single_use(data=asset)
def mean(data):
    import random
    return sum(data)/len(data) + + random.random()

@sy.syft_function_single_use(data=asset)
def reveal_data(data):
    return data

In [10]:
# Create a project
new_project = sy.Project(
    name="My Cool Project",
    description="""Hi, I want to calculate the mean of your private data,\
                    pretty please!""",
    members=[guest_client],
)
new_project

syft.service.project.project.ProjectSubmit

In [11]:
new_project.create_code_request(func, guest_client)
new_project.create_code_request(mean, guest_client)
new_project.create_code_request(reveal_data, guest_client)

<class 'syft.service.response.SyftSuccess'>: Request added successfully

In [12]:
project = new_project.start()
project

syft.service.project.project.Project

## Messaging

### Check New Messages

In [13]:
admin_client.notifications

If possible try using client.requests/client.projects


### Send a Message

### Mark as Read or Unread

In [14]:
message = admin_client.notifications[0]
message

If possible try using client.requests/client.projects


```python
class Message:
  id: str = d2623ec33db04b5abf3ec9acfca96ad3
  subject: str = "Code Request for Project: My Cool Project has been submitted by jane@caltech.edu"
  status: str = MessageStatus.UNREAD
  created_at: str = 2023-06-20 05:09:47
  linked_obj: str = <<class 'syft.service.request.request.Request'>: 020b001795724937b1331ff16dd52d83@<Node: 9458ac6a16464e7bb6211f87d19897dd>

```

In [15]:
message.mark_read()

```python
class Message:
  id: str = d2623ec33db04b5abf3ec9acfca96ad3
  subject: str = "Code Request for Project: My Cool Project has been submitted by jane@caltech.edu"
  status: str = MessageStatus.READ
  created_at: str = 2023-06-20 05:09:47
  linked_obj: str = <<class 'syft.service.request.request.Request'>: 020b001795724937b1331ff16dd52d83@<Node: 9458ac6a16464e7bb6211f87d19897dd>

```

In [16]:
admin_client.notifications

If possible try using client.requests/client.projects


In [17]:
message.mark_unread()

```python
class Message:
  id: str = d2623ec33db04b5abf3ec9acfca96ad3
  subject: str = "Code Request for Project: My Cool Project has been submitted by jane@caltech.edu"
  status: str = MessageStatus.UNREAD
  created_at: str = 2023-06-20 05:09:47
  linked_obj: str = <<class 'syft.service.request.request.Request'>: 020b001795724937b1331ff16dd52d83@<Node: 9458ac6a16464e7bb6211f87d19897dd>

```

In [18]:
admin_client.notifications

If possible try using client.requests/client.projects


## Requests

### Approving

In [19]:
admin_client.requests

In [20]:
func_request = admin_client.requests[-3]
func_request

syft.service.request.request.Request

In [21]:
func_request.approve()

<class 'syft.service.response.SyftSuccess'>: Request 020b001795724937b1331ff16dd52d83 changes applied

In [22]:
admin_client.requests

### Substituting

In [23]:
mean_request = admin_client.requests[-2]
mean_request

syft.service.request.request.Request

In [24]:
admin_asset = admin_client.datasets[0].assets[0]

In [25]:
result = mean_request.code.unsafe_function(data=admin_asset)
result

1.3277902827719106

In [26]:
mean_request.accept_by_depositing_result(result)
mean_request

syft.service.request.request.Request

In [30]:
admin_client.projects[0].requests

[syft.service.request.request.Request,
 syft.service.request.request.Request,
 syft.service.request.request.Request]

### Rejecting

In [36]:
dangerous_request = admin_client.projects[0].requests[-1]
dangerous_request

syft.service.request.request.Request

In [39]:
dangerous_request.code

```python
class UserCode:
  id: str = 9ef2fa3dc3b84e1eb225c690889cfc92
  status.approved: str = False
  service_func_name: str = "reveal_data"

```

In [40]:
dangerous_request.deny(reason="Dangerous request, you just want to reveal the data!")

<class 'syft.service.response.SyftSuccess'>: Request 9a358b344e3c4ba693fd26a0c47708b8 successfully denied !

In [41]:
admin_client.projects[0]

syft.service.project.project.Project